In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.opts.defaults(hv.opts.Curve(width=500), 
                 hv.opts.HLine(alpha=0.5, color='r', line_dash='dashed'))

In [ ]:
import random
import numpy as np
import scipy.stats

# Caminata aleatoria (random walk)


<img src="images/drunkards-walk.png" width="500">



## Introducción

Una caminata aleatoria es una secuencia de pasos o movimientos que se escogen aleatoriamente dentro de un cierto espacio.

Considere por ejemplo el espacio de los enteros en una dimensión

<img src="images/1d_integer.png" width="500">

1. Digamos que inicialmente estamos en la posición $0$
1. En cada paso lanzamos una moneda
1. Si sale cara nos movemos a la derecha $+1$, si sale sello nos movemos a la izquierda $-1$
1. ¿Dónde estoy luego de lanzar $r$ veces la moneda?

Podemos definir la posición luego de lanzar $i+1$ veces la moneda como

$$
p_{i+1} = p_{i} + 2 \cdot m -1
$$

donde $m \in \{0, 1\}$ es una variable aleatoria que se distribuye Bernoulli

$$
m \sim \text{Bernoulli}(0.5)
$$

y estamos asumiendo una moneda "justa", es decir con probabilidad $0.5$

:::{important}

Esta distribución define las "transiciones" o pasos de mi caminata

:::

Más adelante veremos como responder una pregunta como la planteada utilizando una simulación de Monte Carlo 

### Aplicaciones de las caminatas aleatorias

La caminata aleatoria es un modelo ampliamente utilizando con aplicaciones en ecología, física, economía, entre otros. A continuación algunos ejemplos

- [Comportamiento de un animal](https://core.ac.uk/download/pdf/36054338.pdf)
- [Movimientos realizados por las particulas de un gas](https://en.wikipedia.org/wiki/Brownian_motion)
- [La deriva genética](https://es.wikipedia.org/wiki/Deriva_gen%C3%A9tica)
- El precio de las acciones de una firma


En la actividad asociada a esta lección veremos como podemos aplicarlo en un contexto económico. 

La clave de este tipo de modelo está en el modelamiento de la **probabilidad de transición o paso**

### Proceso estocástico y propiedad de Markov

Una caminata aleatoria es un tipo particular de **proceso estocástico**. Llamamos proceso estocástico a una colección de variables aleatorias ordenadas como una secuencia

Por ejemplo

$$
\{X(t): t \in \mathbb{R}^+\}
$$

es una secuencia de variables aleatorias indexadas en el tiempo o

$$
\{X_n: n \in \mathbb{Z}\}
$$

es una secuencia de variables aleatorias con un índice discreto

Una caminata aleatoria es además un caso particular de **proceso Markoviano**. La propiedad de Markov dice que el lugar donde estaré en el futuro sólo depende de donde estoy en el presente. No hay dependencia del pasado. Esto se resume matematicamente como:

$$
P(X_{t+1}|X_{t}, X_{t-1}, \ldots, X_{1}) = P(X_{t+1}|X_{t})
$$

En una próxima lección estudiaremos en detalle las **cadenas de Markov**

## Ejemplo: Caminata en dos dimensiones

Sea un espacio discreto en dos dimensiones y un personaje que parte en la posición $(0, 0)$. 

- El personaje puede moverse hacia el Norte, Sur, Este o Oeste. 
- La probabilidad de escoger cada una de estas direcciones es $1/4$

En promedio 

> ¿A qué distancia del centro estará el personaje luego de $10$, $100$ y $1000$ pasos?

Respondamos esta pregunta usando una simulación de Monte Carlo. 

Primero escribimos una función que recibe una cantidad de pasos y retorna una arreglo con las posiciones del personaje en cada instante. La decisión aleatoria de la dirección la tomaremos en usando la función `np.random.choice`

In [ ]:
def caminata_bidimensional(pasos):
    probs = np.array([1/4, 1/4, 1/4, 1/4])
    direcciones = np.array([[0, 1], [1, 0], [-1, 0], [0, -1]])
    posicion = np.zeros(shape=(pasos+1, 2))   
    decisiones = np.random.choice(len(probs), size=pasos, p=probs)
    desplazamientos = direcciones[decisiones, :]
    posicion[1:, :] = np.cumsum(desplazamientos, axis=0)
    return posicion

La siguiente gráfica muestra el recorrido y destino final del personaje

In [ ]:
np.random.seed(0)
pos = caminata_bidimensional(100)
p_recorrido = hv.Curve((pos[:, 0], pos[:, 1])).opts(xlim=(-15, 15), ylim=(-15, 15))
p_destino = hv.Points((pos[-1, 0], pos[-1, 1])).opts(size=10)
hv.Overlay([p_recorrido, p_destino])

Respondamos ahora la pregunta usando Montecarlo. Utilizaremos la [distancia de Manhattan](https://es.wikipedia.org/wiki/Geometr%C3%ADa_del_taxista) para medir la distancia al centro del mapa

In [ ]:
def distancia_manhattan(posicion):
    return np.sum(np.absolute(posicion[-1, :]))

repeticiones = 1000
print("En promedio...")
for pasos in [10, 100, 1000]:
    distancias = np.empty(shape=(repeticiones,))
    for i in range(repeticiones):
        distancias[i] = distancia_manhattan(caminata_bidimensional(pasos))
    print(f"...luego de {pasos} pasos, el personaje está a {np.mean(distancias)} del centro del mapa")    

¿Cómo cambia el resultado con el número de repeticiones para el caso de 100 pasos?

In [ ]:
Ns = np.logspace(0, 4, 10)
stats = np.zeros(shape=(len(Ns), 3))

pasos = 100
for k, repeticiones in enumerate(Ns):
    repeticiones = int(repeticiones)
    distancias = np.empty(shape=(repeticiones,))
    for i in range(repeticiones):
        distancias[i] = distancia_manhattan(caminata_bidimensional(pasos))
    stats[k, :] = np.percentile(distancias, (25, 50, 75))   

La siguiente gráfica muestra la mediana y el rango intercuartial de las simulaciones a medida que aumentan las repeticiones. La linea roja puntada corresponde a la raíz cuadrada del número de pasos

In [ ]:
p_med = hv.Curve((Ns, stats[:, 1]), 'Repeticiones', 'Distancia al centro', label='Mediana').opts(logx=True)
p_iqr = hv.Area((Ns, stats[:, 0], stats[:, 2]), vdims=['y', 'y2'], label='IQR').opts(alpha=0.25)

hv.Overlay([p_med, p_iqr, hv.HLine(np.sqrt(pasos))])


## Actividad formativa: Jugando a la ruleta (la casa siempre gana)

<img src="images/roulette.png" width="500">

Sea un casino con una ruleta con 36 casilleros numerados del 1 al 36

Una persona llega con al casino y comienza a jugar ruleta. Su estrategia de juego es siempre 

- apostarle $1.000$ pesos a los números pares si el resultado anterior fue impar
- apostarle $1.000$ pesos a los números impares si el resultado anterior fue par

En promedio:

> ¿Cuánto podría ganar/perder esta persona con su particular estrategia?

Utilizemos una simulación de Monte Carlo para resolverlo. En primer lugar escribiremos una clase `Ruleta` con los métodos `girar` y `apostar`. Para seleccionar un casillero de forma aleatoria utilizaremos la función `choice` de la librería `random` de Python

In [ ]:
class Ruleta():
    def __init__(self):
        self.casilleros = []
        for i in range(1, 36+1):
            self.casilleros.append(i)
        
    def _girar(self):
        return random.choice(self.casilleros)
    
    def apostar(self, apuestas, cantidad):
        bola = self._girar()
        if bola in apuestas:
            ganancia = 36/len(apuestas)            
        else:
            ganancia = 0
        return cantidad*(ganancia - 1), bola

Simulemos 10 sesiones de juego bajo las condiciones anteriores. Las lineas de colores son la ganancia en cada una de las sesiones. La linea negra marca el promedio de las sesiones. 

In [ ]:
random.seed(12345)
casino_dreams = Ruleta()  
simulaciones = []
giros = 1000
dinero_apuesta = 1_000
pares = [x for x in range(2, 37, 2)]
impares = [x for x in range(1, 37, 2)]


for k in range(10):
    retornos = np.zeros(shape=(giros,))
    # La primera apuesta es a par
    retornos[0], bola = casino_dreams.apostar(pares, dinero_apuesta)
    # Las apuestas siguientes dependen del resultado anterior de la ruleta
    for i in range(1, giros):
        if bola in pares:
            retornos[i], bola = casino_dreams.apostar(impares, dinero_apuesta)
        else:
            retornos[i], bola = casino_dreams.apostar(pares, dinero_apuesta)
            
    simulaciones.append(np.cumsum(retornos))
simulaciones = np.vstack(simulaciones)

In [ ]:
p = []
for retornos_acumulados in simulaciones:
    p.append(hv.Curve((retornos_acumulados), 'Apuestas', 'Retorno acumulado [$]'))
hv.Overlay(p) * hv.HLine(0) * hv.Curve((np.mean(simulaciones, axis=0))).opts(color='k')

:::{note}

En promedio el retorno está en torno a cero. Al parecer la estrategia no es muy efectiva.

:::

Una ruleta de casino tiene además dos casilleros de color verde denominados "0" y "00". Si la bola cae en uno de estos casilleros la casa se lleva todo. ¿Cómo cambia el resultado anterior si agregamos estos casilleros?

In [ ]:
class RuletaRealista(Ruleta):
    
    def __init__(self):
        super().__init__()
        self.casilleros.append(0) # Representa el cero
        self.casilleros.append(-1) # Represeta el doble cero

random.seed(12345)
casino_dreams = RuletaRealista()  
simulaciones = []
giros = 1000
dinero_apuesta = 1_000
pares = [x for x in range(2, 37, 2)]
impares = [x for x in range(1, 37, 2)]

for k in range(10):
    retornos = np.zeros(shape=(giros,))
    # La primera apuesta es a par
    retornos[0], bola = casino_dreams.apostar(pares, dinero_apuesta)
    # Las apuestas siguientes dependen del resultado anterior de la ruleta
    for i in range(1, giros):
        if bola in pares:
            retornos[i], bola = casino_dreams.apostar(impares, dinero_apuesta)
        else:
            retornos[i], bola = casino_dreams.apostar(pares, dinero_apuesta)
            
    simulaciones.append(np.cumsum(retornos))
simulaciones = np.vstack(simulaciones)

In [ ]:
p = []
for retornos_acumulados in simulaciones:
    p.append(hv.Curve((retornos_acumulados), 'Apuestas', 'Retorno acumulado [$]'))
hv.Overlay(p) * hv.HLine(0) * hv.Curve((np.mean(simulaciones, axis=0))).opts(color='k')

:::{note}

El promedio es claramente inferior a cero, con estos nuevos casilleros en promedio "la casa gana"

:::

Reflexione acerca de la estrategia que utilizó el jugador. Luego revise la [falacia del apostador](https://es.wikipedia.org/wiki/Falacia_del_apostador)

## Propiedades estadísticas del método de Monte Carlo

Como vimos en la lección anterior la integración por Monte Carlo consiste en estimar el valor esperado de una función $g(x)$, definido como

$$
\mathbb{E}[g(X)] = \int g(x) f(x) \,dx
$$

utilizando la siguiente aproximación en base a muestras

$$
\hat g_N = \frac{1}{N} \sum_{i=1}^N g(x_i) \quad x_i \sim f(x)
$$

donde $f(x)$ es la función de densidad de probabilidad de $x$.

La distribución asintótica de este estimador es

$$
\hat g_N \sim \mathcal{N} \left( \mathbb{E}[g(X)], \frac{\sigma_N^2}{N} \right)
$$

donde la varianza muestreal se puede estimar como

$$
\sigma_N^2 = \frac{1}{N-1} \sum_{i=1}^N (g(x_i) - \hat g_N)^2 
$$

:::{important}

El estimador converge a su valor esperado a una tasa de $\frac{1}{\sqrt{N}}$ 

:::

Lo anterior es consecuencia de la Ley Fuerte de los grandes números y del Teorema central del límite

:::{note}

El estimador converge sin importar cual sea la dimensionalidad de la integral. El método de Monte Carlo es entonces una alternativa para resolver integrales complicadas. Volveremos a este punto en una lección siguiente.

:::

### Ley fuerte de los grandes números 

Sean $X_1, X_2, \ldots, X_N$ variables aleatorias (V.A.) independientes e idénticamente distribuidas (i.i.d.) con 

$$
\mathbb{E}[X_i] = \mu
$$

Entonces se cumple que su promedio 

$$ 
\bar X = \frac{1}{N} (X_1 + X_2 + \ldots + X_N) \to \mu
$$

cuando $N \to \infty$. Es decir que el promedio converge al valor esperado con N grande.



La ley fuerte nos dice que podemos aproximar $\mu$ con $\bar X$ pero no nos da pistas sobre que tan cerca está $\bar X$ de $\mu$. Esto último es importante puesto que en la práctica $N$ nunca será $\infty$


### Teorema central del límite

Si $X_1, X_2, \ldots, X_N$ son V.A i.i.d., entonces su promedio de distribuye como

$$ 
\bar X \sim \mathcal{N}\left(\mu, \frac{\sigma^2}{N} \right) 
$$

cuando $N \to \infty$. Es decir que cuando N es grande el promedio (suma) se distribuye como una normal centrada en $\mu$ y con desviación estándar $\frac{\sigma}{\sqrt{N}}$

Esto ocurre sin importar cual sea la distribución original de las V.A., siempre y cuando se cumpla el supuesto i.i.d.

Más importante aun: La tasa a la que converge el estimador a su valor esperado es $\frac{1}{\sqrt{N}}$


### Ejemplo: La distribución promedio de lanzar $N$ dados

La siguiente gráfica interactiva ejemplifica como el promedio de VAs con distribución multinomial tiende a una normal (y se concentra) a medida que $N$ crece

In [ ]:
average_dice_histogram = {}
for number_of_dices in [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 100, 500]:
    dist = scipy.stats.multinomial(n=number_of_dices, p=[1/6]*6)
    repeats = dist.rvs(size=1000)
    average_dice = np.sum(repeats*range(1, 7)/number_of_dices, axis=1)
    average_dice_histogram[number_of_dices] = np.histogram(average_dice, bins=80, density=True, range=(1, 6))

In [ ]:
hMap = hv.HoloMap(kdims='Número de dados')
for number_of_dices, (bins, edges) in average_dice_histogram.items():
    
    hMap[number_of_dices] = hv.Histogram((edges, bins), 
                                         kdims='Promedio de los dados', 
                                         vdims='Densidad').opts(width=500)
hMap